In [1]:
!pip install -q transformers==4.44.2 datasets torch seaborn protobuf==3.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
import numpy as np

2025-12-07 16:27:57.459546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765124877.641159      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765124877.694493      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv('/kaggle/input/cogdis-data/train.csv')
print("Tổng số mẫu:", len(df))
df.head()

Tổng số mẫu: 6440


,original_text,label,bias_type,dialogue,explanation
0,Tôi không chắc mình đang mắc phải loại rối loạ...,1,Labeling and Mislabeling,A: Mình thấy như đang chết dần với mớ vấn đề n...,Người nói tự gán nhãn bản thân là 'kẻ thất bại...
1,Tôi đã la mắng con mình vì quá hiếu động trong...,1,Overgeneralization,A: Nãy mình lỡ la nhóc vì nó quậy quá lúc kể c...,Người nói biến một hành vi xảy ra trong một tì...
2,"Tôi chỉ là người làm theo hướng dẫn thôi, khôn...",1,Magnification and Minimization,A: Sản phẩm này mình chỉ làm theo hướng dẫn th...,Người nói hạ thấp công sức và kỹ năng của bản ...
3,Mất tích cũng không ai tìm,1,Jumping to Conclusions,A: Mình mà mất tích chắc chẳng ai chủ động đi ...,Người nói tự suy đoán tiêu cực rằng không ai q...
4,Mình không nên bỏ cuộc dễ dàng như vậy.,1,Should Statements,A: Mình bỏ dự án rồi… Nhưng nghĩ lại thấy **kh...,Người nói tự áp đặt chuẩn mực cứng nhắc về sự ...


In [4]:
# 4. Tách dữ liệu train/test
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])

# 5. Tokenizer PhoBERT
MODEL_NAME = "vinai/phobert-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
class BiasDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

In [6]:
# Stage 1: Label Classification
train_dataset_label = BiasDataset(
    train_df['dialogue'].tolist(),
    train_df['label'].tolist(),
    tokenizer
)
val_dataset_label = BiasDataset(
    val_df['dialogue'].tolist(),
    val_df['label'].tolist(),
    tokenizer
)

# Stage 2: Bias Type Classification (Chỉ lấy dữ liệu label=1 TỪ TẬP ĐÃ CHIA)

# 1. Chuẩn bị tập Train Stage 2
# Lọc các mẫu có label=1 từ tập train Stage 1
train_b = train_df[train_df['label'] == 1].copy()

# 2. Chuẩn bị tập Validation Stage 2
# Lọc các mẫu có label=1 từ tập validation Stage 1
val_b = val_df[val_df['label'] == 1].copy()

print("\n--- Dữ liệu Stage 2 ---")
print("Số mẫu Train Stage 2 (label=1):", len(train_b))
print("Số mẫu Validation Stage 2 (label=1):", len(val_b))

# Mã hóa bias_type (sử dụng trên toàn bộ dữ liệu có label=1 để đảm bảo ánh xạ)
df_stage2_all = df[df['label'] == 1].copy()
bias_type2id = {b: i for i, b in enumerate(sorted(df_stage2_all['bias_type'].unique()))}
id2bias_type = {v: k for k, v in bias_type2id.items()}

# Áp dụng mã hóa cho tập train và validation Stage 2
train_b['bias_type_id'] = train_b['bias_type'].map(bias_type2id)
val_b['bias_type_id'] = val_b['bias_type'].map(bias_type2id)

print("Các loại sai lệch:", bias_type2id)
print("Phân bố lớp trong Train Stage 2:\n", train_b['bias_type'].value_counts())

train_dataset_bias = BiasDataset(
    train_b['dialogue'].tolist(),
    train_b['bias_type_id'].tolist(),
    tokenizer)
val_dataset_bias = BiasDataset(
    val_b['dialogue'].tolist(),
    val_b['bias_type_id'].tolist(),
    tokenizer)


--- Dữ liệu Stage 2 ---
Số mẫu Train Stage 2 (label=1): 5342
Số mẫu Validation Stage 2 (label=1): 593
Các loại sai lệch: {'All-or-Nothing Thinking': 0, 'Disqualifying the Positive': 1, 'Emotional Reasoning': 2, 'Jumping to Conclusions': 3, 'Labeling and Mislabeling': 4, 'Magnification and Minimization': 5, 'Mental Filter': 6, 'Overgeneralization': 7, 'Personalization': 8, 'Should Statements': 9}
Phân bố lớp trong Train Stage 2:
 bias_type
Labeling and Mislabeling          731
Disqualifying the Positive        636
Jumping to Conclusions            634
All-or-Nothing Thinking           631
Emotional Reasoning               580
Magnification and Minimization    447
Personalization                   429
Mental Filter                     423
Overgeneralization                418
Should Statements                 413
Name: count, dtype: int64


In [7]:
# 7. Training function with HuggingFace Trainer
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    report = classification_report(labels, preds, target_names=[str(l) for l in range(len(set(labels)))])
    return {'f1': f1, 'report': report}

# def compute_metrics(pred):
#     labels = pred.label_ids
#     logits = pred.predictions[0]  # <-- FIX CHO BARTpho
#     preds = logits.argmax(-1)

#     f1 = f1_score(labels, preds, average="weighted")
#     return {"f1": f1}


In [8]:
# Stage 1 Model: Label classifier
model_label = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

training_args_label = TrainingArguments(
    output_dir='./results_label',
    eval_strategy='epoch',
    save_strategy='no',            # ❌ Không lưu checkpoint
    save_total_limit=0,
    logging_strategy='steps',      # log theo bước
    logging_steps=100,  
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs_label',
    load_best_model_at_end=False,
    metric_for_best_model='f1',
    report_to=[],
)

trainer_label = Trainer(
    model=model_label,
    args=training_args_label,
    train_dataset=train_dataset_label,
    eval_dataset=val_dataset_label,
    compute_metrics=compute_metrics
)


pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
trainer_label.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Report
1,0.336800,0.276809,0.882844,precision recall f1-score support 0 0.00 0.00 0.00 51 1 0.92 1.00 0.96 593 accuracy 0.92 644 macro avg 0.46 0.50 0.48 644 weighted avg 0.85 0.92 0.88 644
2,0.263900,0.277142,0.882844,precision recall f1-score support 0 0.00 0.00 0.00 51 1 0.92 1.00 0.96 593 accuracy 0.92 644 macro avg 0.46 0.50 0.48 644 weighted avg 0.85 0.92 0.88 644
3,0.262000,0.280331,0.882844,precision recall f1-score support 0 0.00 0.00 0.00 51 1 0.92 1.00 0.96 593 accuracy 0.92 644 macro avg 0.46 0.50 0.48 644 weighted avg 0.85 0.92 0.88 644


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_funct

TrainOutput(global_step=1089, training_loss=0.28794220328002595, metrics={'train_runtime': 1914.3733, 'train_samples_per_second': 9.083, 'train_steps_per_second': 0.569, 'total_flos': 8102211272773632.0, 'train_loss': 0.28794220328002595, 'epoch': 3.0})

In [10]:
trainer_label.evaluate()

# pred_out = trainer_label.predict(val_dataset_label)
# logits = pred_out.predictions[0]
# y_pred = logits.argmax(-1)

# print("\n================ Classification Report ================")
# print(classification_report(pred_out.label_ids, y_pred))

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.28033071756362915,
 'eval_f1': 0.8828436861370679,
 'eval_report': '              precision    recall  f1-score   support\n\n           0       0.00      0.00      0.00        51\n           1       0.92      1.00      0.96       593\n\n    accuracy                           0.92       644\n   macro avg       0.46      0.50      0.48       644\nweighted avg       0.85      0.92      0.88       644\n',
 'eval_runtime': 24.6406,
 'eval_samples_per_second': 26.136,
 'eval_steps_per_second': 1.664,
 'epoch': 3.0}

In [11]:
# Stage 2 Model: Bias type classifier
model_bias = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(bias_type2id))

training_args_bias = TrainingArguments(
    output_dir='./results_bias',
    eval_strategy='epoch',
    save_strategy='no',            # ❌ Không lưu checkpoint
    save_total_limit=0,
    logging_strategy='steps',      # log theo bước
    logging_steps=100,  
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs_bias',
    load_best_model_at_end=False,
    metric_for_best_model='f1',
    report_to=[],
)

trainer_bias = Trainer(
    model=model_bias,
    args=training_args_bias,
    train_dataset=train_dataset_bias,
    eval_dataset=val_dataset_bias,
    compute_metrics=compute_metrics
)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
trainer_bias.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Report
1,0.310500,0.229042,0.943655,precision recall f1-score support 0 0.97 0.99 0.98 77 1 0.84 0.91 0.87 75 2 0.98 0.98 0.98 58 3 0.95 0.99 0.97 74 4 0.96 0.95 0.96 81 5 0.94 0.74 0.83 46 6 0.90 0.98 0.94 45 7 1.00 0.89 0.94 37 8 0.96 0.96 0.96 54 9 0.98 1.00 0.99 46 accuracy 0.94 593 macro avg 0.95 0.94 0.94 593 weighted avg 0.95 0.94 0.94 593
2,0.134400,0.180617,0.952540,precision recall f1-score support 0 0.99 0.99 0.99 77 1 0.88 0.89 0.89 75 2 0.97 1.00 0.98 58 3 0.96 0.97 0.97 74 4 0.97 0.95 0.96 81 5 0.88 0.83 0.85 46 6 0.94 0.98 0.96 45 7 0.97 0.95 0.96 37 8 0.96 0.96 0.96 54 9 1.00 1.00 1.00 46 accuracy 0.95 593 macro avg 0.95 0.95 0.95 593 weighted avg 0.95 0.95 0.95 593
3,0.055300,0.206443,0.954069,precision recall f1-score support 0 0.99 0.97 0.98 77 1 0.87 0.95 0.90 75 2 0.97 1.00 0.98 58 3 0.96 0.97 0.97 74 4 0.96 0.95 0.96 81 5 0.97 0.78 0.87 46 6 0.94 0.98 0.96 45 7 0.97 0.95 0.96 37 8 0.96 0.96 0.96 54 9 1.00 1.00 1.00 46 accuracy 0.95 593 macro avg 0.96 0.95 0.95 593 weighted avg 0.96 0.95 0.95 593


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1002, training_loss=0.3914194528753321, metrics={'train_runtime': 1775.1211, 'train_samples_per_second': 9.028, 'train_steps_per_second': 0.564, 'total_flos': 7467767865022464.0, 'train_loss': 0.3914194528753321, 'epoch': 3.0})

In [13]:
trainer_bias.evaluate()

# pred_out = trainer_bias.predict(val_dataset_bias)
# logits = pred_out.predictions[0]
# y_pred = logits.argmax(-1)

# print("\n================ Classification Report ================")
# print(classification_report(pred_out.label_ids, y_pred))

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.20644298195838928,
 'eval_f1': 0.9540693495817587,
 'eval_report': '              precision    recall  f1-score   support\n\n           0       0.99      0.97      0.98        77\n           1       0.87      0.95      0.90        75\n           2       0.97      1.00      0.98        58\n           3       0.96      0.97      0.97        74\n           4       0.96      0.95      0.96        81\n           5       0.97      0.78      0.87        46\n           6       0.94      0.98      0.96        45\n           7       0.97      0.95      0.96        37\n           8       0.96      0.96      0.96        54\n           9       1.00      1.00      1.00        46\n\n    accuracy                           0.95       593\n   macro avg       0.96      0.95      0.95       593\nweighted avg       0.96      0.95      0.95       593\n',
 'eval_runtime': 23.0383,
 'eval_samples_per_second': 25.74,
 'eval_steps_per_second': 1.649,
 'epoch': 3.0}

In [14]:
# 9. Lưu mô hình

# Lưu Stage 1: Label classifier
model_label.save_pretrained('./saved_model_label')
tokenizer.save_pretrained('./saved_model_label')

# Lưu Stage 2: Bias type classifier
model_bias.save_pretrained('./saved_model_bias')
tokenizer.save_pretrained('./saved_model_bias')

print("Lưu mô hình hoàn tất!")

Lưu mô hình hoàn tất!


In [15]:
!zip -r /kaggle/working/saved_model_label.zip /kaggle/working/saved_model_label
!zip -r /kaggle/working/saved_model_bias.zip /kaggle/working/saved_model_bias

  adding: kaggle/working/saved_model_label/ (stored 0%)
  adding: kaggle/working/saved_model_label/special_tokens_map.json (deflated 57%)
  adding: kaggle/working/saved_model_label/model.safetensors (deflated 12%)
  adding: kaggle/working/saved_model_label/bpe.codes (deflated 59%)
  adding: kaggle/working/saved_model_label/vocab.txt (deflated 55%)
  adding: kaggle/working/saved_model_label/added_tokens.json (stored 0%)
  adding: kaggle/working/saved_model_label/config.json (deflated 50%)
  adding: kaggle/working/saved_model_label/tokenizer_config.json (deflated 77%)
  adding: kaggle/working/saved_model_bias/ (stored 0%)
  adding: kaggle/working/saved_model_bias/special_tokens_map.json (deflated 57%)
  adding: kaggle/working/saved_model_bias/model.safetensors (deflated 12%)
  adding: kaggle/working/saved_model_bias/bpe.codes (deflated 59%)
  adding: kaggle/working/saved_model_bias/vocab.txt (deflated 55%)
  adding: kaggle/working/saved_model_bias/added_tokens.json (stored 0%)
  adding: 

In [16]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('saved_model_label.zip')
FileLink('saved_model_bias.zip')

/kaggle/working


/kaggle/working/saved_model_bias.zip

In [17]:
# 8. Inference Two-Stage
def predict_two_stage(texts, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Đưa model về device
    model_label.to(device)
    model_bias.to(device)
    model_label.eval()
    model_bias.eval()

    # Stage 1
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model_label(**inputs)
        preds_label = torch.argmax(outputs.logits, dim=1).cpu().numpy()
    
    # Stage 2
    preds_bias = []
    for i, label in enumerate(preds_label):
        if label == 1:  # chỉ dự đoán bias_type nếu label=1
            inputs_stage2 = tokenizer([texts[i]], padding=True, truncation=True, max_length=256, return_tensors='pt').to(device)
            with torch.no_grad():
                outputs2 = model_bias(**inputs_stage2)
                pred_bias = torch.argmax(outputs2.logits, dim=1).item()
            preds_bias.append(list(bias_type2id.keys())[pred_bias])
        else:
            preds_bias.append(None)
    return preds_label, preds_bias

# Ví dụ
texts = ["Tôi cảm thấy mọi thứ đều tồi tệ, không có gì tốt cả."]
pred_label, pred_bias = predict_two_stage(texts)
print(pred_label, pred_bias)

[1] ['Labeling and Mislabeling']


In [18]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [19]:
test_df = pd.read_csv('/kaggle/input/cogdis-data/test.csv')

test_df['bias_id'] = test_df['bias_type'].map(bias_type2id).fillna(-1).astype(int)

In [20]:
class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_label.to(device)
model_bias.to(device)
model_label.eval()
model_bias.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 1024, padding_idx=1)
      (position_embeddings): Embedding(258, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [21]:
dataset1 = TextDataset(test_df['dialogue'].tolist(), test_df['label'].tolist(), tokenizer)
loader1 = DataLoader(dataset1, batch_size=16, shuffle=False)

def evaluate(model, dataloader):
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()
            all_preds.extend(preds)
            if 'labels' in batch:
                all_labels.extend(batch['labels'].numpy())
    return all_preds, all_labels

preds1, labels1 = evaluate(model_label, loader1)

print("== Stage 1: Nhận diện sai lệch ==")
print("Accuracy:", accuracy_score(labels1, preds1))
print("F1 Score:", f1_score(labels1, preds1, average='weighted'))
print(classification_report(labels1, preds1))

== Stage 1: Nhận diện sai lệch ==
Accuracy: 0.9213793103448276
F1 Score: 0.8836775008045151
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        57
           1       0.92      1.00      0.96       668

    accuracy                           0.92       725
   macro avg       0.46      0.50      0.48       725
weighted avg       0.85      0.92      0.88       725



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
mask = (test_df['label'] == 1) & (test_df['bias_id'] != -1)
filtered_texts = test_df.loc[mask, 'dialogue'].tolist()
filtered_labels = test_df.loc[mask, 'bias_id'].tolist()

dataset2 = TextDataset(filtered_texts, filtered_labels, tokenizer)
loader2 = DataLoader(dataset2, batch_size=16, shuffle=False)

preds2, labels2 = evaluate(model_bias, loader2)

# Convert lại thành string
preds2_str = [id2bias_type[p] for p in preds2]
labels2_str = [id2bias_type[l] for l in labels2]

print("== Stage 2: Phân loại loại sai lệch ==")
print("Accuracy:", accuracy_score(labels2_str, preds2_str))
print("F1 Score (macro):", f1_score(labels2_str, preds2_str, average='macro'))
print(classification_report(labels2_str, preds2_str))

== Stage 2: Phân loại loại sai lệch ==
Accuracy: 0.8802395209580839
F1 Score (macro): 0.876035846096962
                                precision    recall  f1-score   support

       All-or-Nothing Thinking       0.78      0.90      0.84        79
    Disqualifying the Positive       0.84      0.96      0.90        80
           Emotional Reasoning       0.90      0.92      0.91        72
        Jumping to Conclusions       0.93      0.94      0.93        80
      Labeling and Mislabeling       0.92      0.86      0.89        91
Magnification and Minimization       0.85      0.71      0.77        55
                 Mental Filter       0.87      0.85      0.86        54
            Overgeneralization       0.88      0.71      0.78        51
               Personalization       0.93      0.96      0.95        54
             Should Statements       0.96      0.92      0.94        52

                      accuracy                           0.88       668
                     macro avg

In [23]:
# ===== TỔNG HỢP KẾT QUẢ =====

# 1) Gán pred_label (stage 1) vào test_df
test_df["pred_label"] = preds1

# 2) Tạo cột pred_bias mặc định
test_df["pred_bias"] = "none"

# 3) Gán kết quả stage 2 vào đúng vị trí những mẫu có label=1
stage2_indices = test_df.index[mask].tolist()
for idx, pb in zip(stage2_indices, preds2_str):
    test_df.at[idx, "pred_bias"] = pb

# 4) Nếu explanation có trong dataset thì giữ, nếu không thì tạo cột rỗng
if "explanation" not in test_df.columns:
    test_df["explanation"] = ""

# 5) Nếu original_text không tồn tại, dùng dialogue làm fallback
if "original_text" not in test_df.columns:
    test_df["original_text"] = test_df["dialogue"]

# 6) Xuất file CSV theo đúng yêu cầu
output_cols = [
    "original_text",
    "label",
    "bias_type",
    "dialogue",
    "explanation",
    "pred_label",
    "pred_bias"
]

test_df[output_cols].to_csv("test_prediction.csv", index=False)

print("Đã xuất file test_prediction.csv thành công!")

Đã xuất file test_prediction.csv thành công!
